In [1]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta
from dateutil.parser import parse
from scipy.io import loadmat

from parcels import FieldSet, ParticleSet, JITParticle, ErrorCode, AdvectionRK4

%matplotlib inline
plt.rcParams['font.size'] = 14

In [2]:

### Local functions
def make_prefix(date, path, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., ./SalishSea/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
    prefix = os.path.join(path, f'SalishSea_1{res}_{datestr}')
    
    return prefix


def DeleteParticle(particle, fieldset, time):
    """Delete particle from OceanParcels simulation to avoid run failure
    """
    
    print(f'Particle {particle.id} lost !! [{particle.lon}, {particle.lat}, {particle.depth}, {particle.time}]')
    particle.delete()


In [3]:

### Load drifters and definitions
# Define paths
paths = {
    'NEMO': './SalishSea/',
    'coords': './grid/coordinates_seagrid_SalishSea201702.nc',
    'mask': './grid/mesh_mask201702.nc',
    'out': './results',
}

In [4]:

# Duration and timestep [s]
duration = timedelta(days=5)
dt = 90

In [5]:
n = 1000   # number of particles
# Define Gaussian point cloud in the horizontal
r = 10000   # radius of particle cloud [m]
deg2m = 111000 * np.cos(50 * np.pi / 180)
var = (r / (deg2m * 3))**2
x_offset, y_offset = np.random.multivariate_normal([0, 0], [[var, 0], [0, var]], n).T
# Set a uniform distribution in depth, from dmin to dmax
dmin = 0.
dmax = 250.
zvals = dmin + np.random.random_sample(n)*(dmax-dmin)

In [9]:
paths['coords']

'./grid/coordinates_seagrid_SalishSea201702.nc'

In [10]:
### Simulations
start = datetime(2023, 1, 1)
# Build filenames
Ulist, Vlist, Wlist = [], [], []
for day in range(duration.days + 3):
    path_NEMO = make_prefix(start + timedelta(days=day), paths['NEMO'])
    print (path_NEMO)
    Ulist.append(path_NEMO + '_grid_U.nc')
    Vlist.append(path_NEMO + '_grid_V.nc')
    Wlist.append(path_NEMO + '_grid_W.nc')

# Load NEMO forcing : note, depth aware but no vertical advection, particles stay at their original depth
filenames = {
    'U': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Ulist},
    'V': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Vlist},
}
variables = {'U': 'vozocrtx', 'V': 'vomecrty'}
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}

field_set = FieldSet.from_nemo(filenames, variables, dimensions)
# is not working, figure out why...

./SalishSea/SalishSea_1h_20230101_20230101
./SalishSea/SalishSea_1h_20230102_20230102
./SalishSea/SalishSea_1h_20230103_20230103
./SalishSea/SalishSea_1h_20230104_20230104
./SalishSea/SalishSea_1h_20230105_20230105
./SalishSea/SalishSea_1h_20230106_20230106
./SalishSea/SalishSea_1h_20230107_20230107
./SalishSea/SalishSea_1h_20230108_20230108


OSError: FieldSet files not found for variable U: {'lon': './grid/coordinates_seagrid_SalishSea201702.nc', 'lat': './grid/coordinates_seagrid_SalishSea201702.nc', 'depth': './SalishSea/SalishSea_1h_20230101_20230101_grid_W.nc', 'data': ['./SalishSea/SalishSea_1h_20230101_20230101_grid_U.nc', './SalishSea/SalishSea_1h_20230102_20230102_grid_U.nc', './SalishSea/SalishSea_1h_20230103_20230103_grid_U.nc', './SalishSea/SalishSea_1h_20230104_20230104_grid_U.nc', './SalishSea/SalishSea_1h_20230105_20230105_grid_U.nc', './SalishSea/SalishSea_1h_20230106_20230106_grid_U.nc', './SalishSea/SalishSea_1h_20230107_20230107_grid_U.nc', './SalishSea/SalishSea_1h_20230108_20230108_grid_U.nc']}

In [ ]:

# Set output file name.  Maybe change for each run
fn = f'vicente_drifters' + '_'.join(d.strftime('%Y%m%d') for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

In [ ]:

# Execute run
clon, clat = -123.4, 49.18  # choose horizontal centre of the particle cloud
lon, lat, z = clon + x_offset, clat + y_offset, zvals
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))
pset.execute(
    pset.Kernel(AdvectionRK4), runtime=duration, dt=dt,
    output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
    recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
)

In [ ]:

# this cell will fail, but I seem to need to run it to get the outputfiles from the temp directory into my 
# final outfile
pset.execute(fail=fail)

In [ ]:
#coords.close()
#mask.close()
coords = xr.open_dataset(paths['coords'], decode_times=False)
mask = xr.open_dataset(paths['mask'])

In [ ]:
ds = xr.open_dataset(outfile)

In [ ]:
print(ds)

In [ ]:
fig, ax = plt.subplots(figsize=(19, 8))
ax.contourf(coords.nav_lon, coords.nav_lat, mask.tmask[0, 0, ...], levels=[-0.01, 0.01], colors='lightgray')
ax.contour(coords.nav_lon, coords.nav_lat, mask.tmask[0, 0, ...], levels=[-0.01, 0.01], colors='k')
ax.set_xlim([-124., -122.7])
ax.set_ylim([48.55, 49.8])
ax.set_aspect(5/4.4)
nmin, nmax = 0, -1
for traj in range(n):
    s = ax.scatter(ds.lon[traj, nmin:nmax], ds.lat[traj, nmin:nmax])

In [ ]:
nmax = -1
fig, ax = plt.subplots(figsize=(10, 10))
for traj in range(n):
    ax.plot(ds.lon[traj, nmax], -ds.z[traj, nmax], 'o')
ax.grid()

In [ ]:
ds.close()